In [1]:
# import os
# import numpy as np
# import pandas as pd
# import altair as alt
# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib_inline.backend_inline
# matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import DataFrames as DF

In [ ]:
years = ["2017", "2018", "2019", "2020","2021"]

folder_weather = "../dataset/data/weather"
files_weather = readdir(folder_weather)

year = "2017"
files_weather_selected = []
for file in files_weather
    if occursin(year, file)
        push!(files_weather_selected, file)
    end
end
# files_weather_selected

df_weather = pd.DataFrame()
for file in files_weather_selected
    # append(df_weather, pd.read_csv(folder_weather * '/' * file))
    pd.DataFrame(pd.concat([df_weather, pd.read_csv(folder_weather * '/' * file)]))
end

In [ ]:
years = ["2017", "2018", "2019", "2020","2021"]
for year in years:

    folder_weather = r"C:\Users\Mehmet B Batukan\OneDrive - University of Toronto\Documents\Software\github\BikeShareToronto\dataset\data\weather"
    files_weather = os.listdir(folder_weather)

    files_weather_selected = []
    for file in files_weather:
        if year in file:
            files_weather_selected.append(file)
    

    df_weather = pd.DataFrame()
    for file in files_weather_selected:
        df_weather = df_weather.append(pd.read_csv(folder_weather + '\\' + file))

    %cd -q "C:\Users\Mehmet B Batukan\OneDrive - University of Toronto\Documents\Software\github\BikeShareToronto\project"

    # df_weather[["Date", "TimeNew"]] = df_weather["Date/Time"].str.split(' ',expand=True)

    plt.figure()
    # palette = sns.color_palette("Paired", 12)
    palette = sns.diverging_palette(250, 15, s=100, l=50, n=9, center="light")
    ax = sns.boxplot(x="Month", y="Temp (°C)", data=df_weather, palette=palette)
    plt.grid(linewidth= 0.2)
    ax.set_axisbelow(True)
    ax.set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
    plt.title(f"Monthly Weather in Toronto ({year})")
    plt.savefig(f"weather_{year}.svg", dpi=1200, format="svg")